# 2A.eco - Web Scrapping et API avec Pocket

Le notebook revient sur le webscrapping et l'utilisation d'API avec [pocket](https://getpocket.com/).

In [ ]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

## Objectifs des prochaines séances

Connaissez-vous l'application [Pocket](https://getpocket.com/) ? C'est une application qui simplifie le bookmarking. Elle prend la forme d'une extension Chrome / Firefox. Quand on tombe sur un site intéressant, on peut le bookmaker, et ajouter, ou non, des tags pour "qualifier" le contenu.

Cette application répond au besoin de conserver le contenu web pertinent et de le classer.

Au cours des prochaines séances, nous allons construire un outil de machine learning qui :
- se connecte à un compte pocket
- récupère les sites bookmarqués et les tags éventuels
- à partir des articles taggés, prédit les meilleurs tags des sites non-taggés
- tag les articles non taggés

Bref, nous allons concevoir un programme de classification automatique des articles !

## Objectif de la séance

- Créer un compte Pocket
- S'authentifier auprès de l'API
- Populer le compte avec des données via l'API
- Récupérer les données via l'API
- Scraper les sites bookmarqués pour enrichir les données

## Mais c'est quoi une API ?

On vous explique tout ici :  
- [Définition](http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx/notebooks/TD2A_eco_les_API.html#definition)
- [Les API qui existent](http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx/notebooks/TD2A_eco_les_API.html#les-api-qui-existent)
- [Comment parler à une API ?](http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx/notebooks/TD2A_eco_les_API.html#comment-parler-a-une-api)

Donc un API est une interface permettant de _communiquer_ avec une application. En général, on veut récupérer des données. Donc la communication consiste à envoyer une requete HTTP (le plus souvent GET ou POST) et à récupérer des données, souvent au format json. Ici nous voulons récupérer des données d'un compte utilisateur pocket. 

Pour savoir comment on communique précisément avec l'API de pocket, il n'y a pas de secret : il faut lire la doc de ceux qui l'ont codée. On vous a simplifié un peu les étapes ci-après.

## Création d'un compte Pocket

Créer un compte sur https://getpocket.com/signup?ep=4. Il n'y a pas de vérification d'email, donc vous pouvez mettre un faux mail.

Aller sur la _console developer_ de pocket: https://getpocket.com/developer/apps/index.php

<img src="./images/console_developer_pocket.png" width="450"/>

Cliquer sur CREATE AN APPLICATION

Compléter le formulaire comme suit (vous pouvez changer le nom de l'application et la description)

<img src="./images/screen_consumer_key.png" width="450"/>

Cliquer sur CREATE APPLICATION

In [ ]:
# keyring pour récupérer un mot de passe
import keyring
import os
CONSUMER_KEY = keyring.get_password("web", os.environ["COMPUTERNAME"] + "pocket")
# input pour le demander
# CONSUMER_KEY = input("Insérer ici le CONSUMER KEY de la plateform WEB ")

<img src="./images/screen_consumer_key2.png" width="450"/>

Vous en aurez besoin pour vous connecter à l'API de pocket.

## Authentification

D'abord il faut s'[authentifier](https://getpocket.com/developer/docs/authentication)

Protocole utilisé ici : [OAUTH2](https://tools.ietf.org/html/rfc6749) (très classique). 

<img src="./images/screen_oauth2.png"/>

6 étapes donc avant d'avoir le droit de récupérer les données. De temps en temps, il existe une librairie python. C'est notre cas : https://github.com/tapanpandita/pocket. On va s'en servir pour s'authentifier. Mais pas pour récupérer les données (elle n'est plus à jour pour faire ça).

### Etape 1 : Obtenir un code d'authorisation => get_request_token

In [ ]:
import pocket
from pocket import Pocket

REDIRECT_URI = "http://localhost:8888/notebooks/API%20Pocket.ipynb"
# c'est l'url à laquelle vous allez rediriger l'utilisateur (ici, vous) après que pocket a authentifié l'utilisateur (vous)
REQUEST_TOKEN = Pocket.get_request_token(consumer_key=CONSUMER_KEY, redirect_uri=REDIRECT_URI)
# print(REQUEST_TOKEN)

### Etape 2: Authoriser l'accès

Il faut le faire à chaque exécution du notebook.

In [ ]:
# Enlever les guillemets autour de REQUEST_TOKEN
url = "https://getpocket.com/auth/authorize?request_token={0}&redirect_uri={1}".format("REQUEST_TOKEN", REDIRECT_URI)
print("Aller à l'url : \n" + url)

Aller à l'url : 
https://getpocket.com/auth/authorize?request_token=REQUEST_TOKEN&redirect_uri=http://localhost:8888/notebooks/API%20Pocket.ipynb


<img src="./images/screen_authorization_pocket.png" width="450"/>

Cliquer sur Autoriser.

### Etape 3: Récupérer le token d'accès

In [ ]:
try:
    USER_CREDENTIALS = Pocket.get_credentials(consumer_key=CONSUMER_KEY, code=REQUEST_TOKEN)
except Exception as e:
    print(e)
# print(USER_CREDENTIALS)

In [ ]:
ACCESS_TOKEN = USER_CREDENTIALS['access_token']
# print(ACCESS_TOKEN)

## Chargement de données sur le nouveau compte

Comme vous venez de créer un compte, vous n'avez pas encore d'articles sauvegardés. On vous a préparé un peu moins de 500 articles (format json). Un tiers de ces articles sont taggés (catégorisés). Un article peut comprendre un ou plusieurs tags.

On vous rappelle que l'objectif à termes sera de prédire les meilleurs tags pour les articles non taggés, étant donnés les mots qui caractérisent ces articles (titre, résumé, et ensemble des mots présents dans le html de la page).

### Chargement du fichier json en mémoire

In [ ]:
import json

with open('./images/data_pocket.json') as fp:    
    data = json.load(fp)

On affiche le premier élément.

In [ ]:
from pprint import pprint
keys = list(data.keys())
pprint(data[keys[0]])

{'authors': {'2832761': {'author_id': '2832761',
                         'item_id': '1883956314',
                         'name': 'float',
                         'url': ''}},
 'excerpt': 'Il est impossible d’écrire un programme sans utiliser de '
            'variable. Ce terme désigne le fait d’attribuer un nom ou '
            'identificateur à des informations : en les nommant, on peut '
            'manipuler ces informations beaucoup plus facilement.',
 'favorite': '0',
 'given_title': 'Types et variables du langage python — Programmation avec le '
                'langage Python',
 'given_url': 'http://www.xavierdupre.fr/app/teachpyx/helpsphinx/c_lang/types.html',
 'has_image': '1',
 'has_video': '0',
 'image': {'height': '0',
           'item_id': '1883956314',
           'src': 'http://www.xavierdupre.fr/app/teachpyx/helpsphinx/_images/math/a283b6104f42fc7a8bf845790aa022ac525329f0.png',
           'width': '0'},
 'images': {'1': {'caption': '',
                  'credit': '

### Exercice 1

Chargement des données du json dans le compte nouvellement créé.

Pour communiquer avec une API, il faut envoyer des requêtes HTTP. Ici on veut ajouter les données du fichier json ("given_url" et "tags") dans le compte Pocket.

Que nous dit la doc ? Consulter https://getpocket.com/developer/docs/v3/add.

Par exemple, pour l'ajout d'un seul item, la doc nous donne l'url à laquelle il faut envoyer une requête (https://getpocket.com/v3/add), et la méthode qu'il faut employer. Ici, il s'agit d'une méthode [POST](https://en.wikipedia.org/wiki/POST_(HTTP)).

Pour envoyer une requête en python, il y a plusieurs solutions. Un des plus simples consiste à utiliser la librairie [requests](http://docs.python-requests.org/en/master/user/quickstart/#make-a-request).

A vous de jouer ! Commencez par un seul (par exemple http://docs.python-requests.org/en/master/user/quickstart/ avec les tags "python, requests"), puis si cela a fonctionné, vous pouvez passer 100 items (pas plus, le maximum de requêtes autorisées par l'API est de 500 par heure et par utilisateur).

Pour voir si cela a marché, il suffit d'aller sur votre compte Pocket :)

## Récupération des données disponibles dans l'API

### Exercice 2

Récupérer les urls et les tags des items qui contiennent le tag "python".

C'est par ici : https://getpocket.com/developer/docs/v3/retrieve. A vous de jouer !

### Exercice 3

Récupérer les urls et les titres des items qui contiennent le mot "python" dans le titre ou l'url

Dans le fichier qui servira à la catégorisation automatique, nous allons avoir besoin : de l'url, du titre, de l'extrait, des tags et du contenu.
    
L'api ne permet pas d'accéder au contenu des sites épinglés. Mais nous pouvons le récupérer grâce à l'url.

### Exercice 4

Constituer d'abord un DataFrame avec les champs resolved_url, resolved_title, excerpt, et les tags des items comprenant le terme python.

## Compléter les données avec du webscraping

### Mais c'est quoi le webscraping ? 

On vous explique tout ici :  
- [Définition](http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx/notebooks/TD2A_Eco_Web_Scraping.html#a-eco-web-scraping)
- [Un détour par le Web : comment fonctionne un site ?](http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx/notebooks/TD2A_Eco_Web_Scraping.html#un-detour-par-le-web-comment-fonctionne-un-site)
- [Scrapper avec python](http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx/notebooks/TD2A_Eco_Web_Scraping.html#scrapper-avec-python)

### Beautiful Soup 

Reprenons notre liste de sites sur python et analysons le contenu HTML.

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
from pprint import pprint

#première url de la liste
if 'df' in locals():
    url = df['url'].iloc[0]
    print(url)
else:
    url = "http://www.xavierdupre.fr"

# récupération du contenu (même librairie que pour les requetes api => 
#requests est une librairie qui permet de faire des requetes http, que cela soit pour des api ou du webscraping)

import requests
r = requests.get(url)

#pprint : librairie pour "pretty print" (essayer sans : on voit pas grand chose)
text = r.text if len(r.text) < 1000 else r.text[:1000] + "\n..."
pprint(text)

('<?xml version="1.0" encoding="utf-8"?>\n'
 '<html>\n'
 '<head>\n'
 '<link TYPE="text/css" href="pMenu2.css" rel="stylesheet"/>\n'
 '<title>Xavier DuprÃ©, ENSAE, Microsoft</title>\n'
 '<meta content="xavier duprÃ©, dupre, ENSAE, Microsoft, Bing, Azure ML, '
 'Azure" name="keywords"/>\n'
 '<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>\n'
 '<link rel="shortcut icon" href="pyhome3.ico" />\n'
 '<meta content="" name="description" />\n'
 '</head>\n'
 '<body>\n'
 '\n'
 '<div class="maintitle">\n'
 '<h1>Xavier DuprÃ©\n'
 '<a href="https://www.linkedin.com/in/xavier-dupr%C3%A9-780924"><img '
 'src="blog/documents/linkedin.png" width="15"/></a>\n'
 '<font size="4"><a '
 'href="http://www.xavierdupre.fr/blog/xd_blog_nojs.html">Blog</a></font>\n'
 '<a href="blog/xdbrss.xml"><img '
 'src="blog/documents/feed-icon-16x16.png"/></a>\n'
 '</h1>\n'
 '</div>\n'
 '\n'
 '\n'
 '<div style="position:absolute; top:1%; left:5%; width:120px;">\n'
 '\n'
 '<p>\n'
 '<a style="padding:5px;"

In [ ]:
# on stock le contenu html dans la variable html
html = r.text

# on "parse" le html grâce à la librairie beautiful soup
soup = BeautifulSoup(html, "html5lib")

# c'est plus "joli" encore que pprint et surtout, 
# il y a plein de méthodes pour extraire les informations que l'on souhaite
soup

<!--?xml version="1.0" encoding="utf-8"?--><html><head>
<link href="pMenu2.css" rel="stylesheet" type="text/css"/>
<title>Xavier DuprÃ©, ENSAE, Microsoft</title>
<meta content="xavier duprÃ©, dupre, ENSAE, Microsoft, Bing, Azure ML, Azure" name="keywords"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="pyhome3.ico" rel="shortcut icon"/>
<meta content="" name="description"/>
</head>
<body>

<div class="maintitle">
<h1>Xavier DuprÃ©
<a href="https://www.linkedin.com/in/xavier-dupr%C3%A9-780924"><img src="blog/documents/linkedin.png" width="15"/></a>
<font size="4"><a href="http://www.xavierdupre.fr/blog/xd_blog_nojs.html">Blog</a></font>
<a href="blog/xdbrss.xml"><img src="blog/documents/feed-icon-16x16.png"/></a>
</h1>
</div>


<div style="position:absolute; top:1%; left:5%; width:120px;">

<p>
<a href="app/ensae_teaching_cs/helpsphinx3/index.html" style="padding:5px;"><img src="app/ensae_teaching_cs/helpsphinx/_static/project_ico.png" width="40px"/></

In [ ]:
type(soup)

bs4.BeautifulSoup

_soup_ est une instance de la classe BeautifulSoup (que l'on a importé de la librairie bs4). C'est une représentation du code html avec des méthodes pratiques, telles que __find__. __find__ permet de trouver la première occurence d'une balise html qu'on lui passe. Par exemple le 1er lien de la page :

In [ ]:
soup.find("a")

<a href="https://www.linkedin.com/in/xavier-dupr%C3%A9-780924"><img src="blog/documents/linkedin.png" width="15"/></a>

Si on veut tous les liens, il faut utiliser __findAll__. Cela renvoie une objet qui se comporte comme une liste. On peut itérer dessus, ou facilement le transformer en objet "list" (en faisant list())

In [ ]:
print(type(soup.findAll("a")))
soup.findAll("a")

<class 'bs4.element.ResultSet'>


[<a href="https://www.linkedin.com/in/xavier-dupr%C3%A9-780924"><img src="blog/documents/linkedin.png" width="15"/></a>,
 <a href="http://www.xavierdupre.fr/blog/xd_blog_nojs.html">Blog</a>,
 <a href="blog/xdbrss.xml"><img src="blog/documents/feed-icon-16x16.png"/></a>,
 <a href="app/ensae_teaching_cs/helpsphinx3/index.html" style="padding:5px;"><img src="app/ensae_teaching_cs/helpsphinx/_static/project_ico.png" width="40px"/></a>,
 <a href="app/actuariat_python/helpsphinx/index.html" style="padding:5px;"><img src="app/actuariat_python/helpsphinx/_static/project_ico.png" width="40px"/></a>,
 <a href="app/ensae_projects/helpsphinx/index.html" style="padding:5px;"><img src="app/ensae_projects/helpsphinx/_static/project_ico.png" width="40px"/></a>,
 <a href="app/jupytalk/helpsphinx/index.html#jupytalk" style="padding:5px;"><img src="app/jupytalk/helpsphinx/_static/project_ico.png" width="40px"/></a>,
 <a href="http://lesenfantscodaient.fr/" style="padding:5px;"><img src="http://lesenfants

On peut sélectionner des tags qui ont certains attributs css. Par exemple ici, on peut vouloir seulement les liens internes du site. Ils ont la classe "internal". Les autres "external".

In [ ]:
list(soup.findAll("a", {"class": "internal"}))

[]

Ce n'est pas toujours comme ça, la plupart du temps, il faudra la forme de la valeur de l'attribut href. les liens externes pourront etre identifiés parce qu'ils sont absolus (on indique l'ensemble du lien comme href="https://docs.python.org/3/reference/expressions.html etc. et non pas un lien relatif comme href="../c_exception/exception.html ('..' signifie "le répertoire parent).

On peut lister toutes les balises h1 et h2 de cette page, en une ligne.

In [ ]:
list(soup.findAll({"h1", "h2"}))

[<h1>Xavier DuprÃ©
 <a href="https://www.linkedin.com/in/xavier-dupr%C3%A9-780924"><img src="blog/documents/linkedin.png" width="15"/></a>
 <font size="4"><a href="http://www.xavierdupre.fr/blog/xd_blog_nojs.html">Blog</a></font>
 <a href="blog/xdbrss.xml"><img src="blog/documents/feed-icon-16x16.png"/></a>
 </h1>]

Naviguer vers les enfants

In [ ]:
h1 = soup.find("h1")
child = h1.a if h1 else None
child if child else "vide"

<a href="https://www.linkedin.com/in/xavier-dupr%C3%A9-780924"><img src="blog/documents/linkedin.png" width="15"/></a>

Retrouver le parent

In [ ]:
child.parent

<h1>Xavier DuprÃ©
<a href="https://www.linkedin.com/in/xavier-dupr%C3%A9-780924"><img src="blog/documents/linkedin.png" width="15"/></a>
<font size="4"><a href="http://www.xavierdupre.fr/blog/xd_blog_nojs.html">Blog</a></font>
<a href="blog/xdbrss.xml"><img src="blog/documents/feed-icon-16x16.png"/></a>
</h1>

Récupérer tous les enfants

In [ ]:
children = soup.find("h1").findAll("a")
children

[<a href="https://www.linkedin.com/in/xavier-dupr%C3%A9-780924"><img src="blog/documents/linkedin.png" width="15"/></a>,
 <a href="http://www.xavierdupre.fr/blog/xd_blog_nojs.html">Blog</a>,
 <a href="blog/xdbrss.xml"><img src="blog/documents/feed-icon-16x16.png"/></a>]

Récupérer les attributs des éléments html

In [ ]:
[c.attrs for c in children]

[{'href': 'https://www.linkedin.com/in/xavier-dupr%C3%A9-780924'},
 {'href': 'http://www.xavierdupre.fr/blog/xd_blog_nojs.html'},
 {'href': 'blog/xdbrss.xml'}]

Accéder à la valeur d'un attribut en particulier

In [ ]:
[c.attrs['href'] for c in children]

['https://www.linkedin.com/in/xavier-dupr%C3%A9-780924',
 'http://www.xavierdupre.fr/blog/xd_blog_nojs.html',
 'blog/xdbrss.xml']

Jusqu'ici, on passe systématiquement par une balise html. Comment faire si on veut récupérer les élements qui ont une class css, quelle que soit la balise html ?

In [ ]:
internals = soup.findAll("", {"class": "internal"})
internals

[]

On a presque fini de passer en revue la librairie. Il reste la notion de "sibling", pratique pour parcourir un tableau. nextSibling : permet de récupérer l'élément html suivant et "de même niveau" dans l'arbre (le prochain frère), previousSibling, le précédent. findChildren permet de trouver tous les enfants.

Enfin la méthode "get_text()" pour récupérer le contenu des balises html. A appliquer en dernier ! En effet, une fois appliquer, on perd toute trace à l'arbre html. SI vous voulez aller plus loin, la doc est bien faite : https://www.crummy.com/software/BeautifulSoup/bs4/doc/

### Exercice 5

Récupérer le contenu des code snippets de la page : http://www.xavierdupre.fr/app/teachpyx/helpsphinx/c_lang/types.html

### Exercice 6

Récupérer la 3ème colonne (intitulée "exemples") du tableau des opérateurs : http://www.xavierdupre.fr/app/teachpyx/helpsphinx/c_lang/types.html

Astuce : explorer les CSS selectors : https://www.crummy.com/software/BeautifulSoup/bs4/doc/.

### Exercice 7

Récupérer le contenu des titres (h1, h2, h3, etc.) et des balise p de l'ensemble des liens de notre liste et compter le nombre d'occurences du mot python, et la porportion que cela représente dans l'ensemble des mots